In [1]:
"""
{"2006": {"Flevoland": 
                {"noordoostpolder": "22", 
                 "urk": "32", 
                 "lelystad": "20", 
                  "almere": "23", 
                  "dronten": "21", 
                  "zeewolde": "26"
                  "__provdata": {
                              "min": ["lelystad", "20"], 
                              "max": ["urk", "32"], 
                              "avg": "24"
                            }
          "__countrydata":
                   {"min": "Flevoland",
                    "max": "Zeeland",
                    "avg": "39"}
"""

#var["2006"][["__countrydata"]["min"]]["__provdata"]["min"]
print

In [2]:
import pandas as pd
import numpy as np
import math
import json
import csv
import glob
import os


def list_columns(year, location):
	"""
	Returns all columns of a specific year.csv file.
	This file is identical to the .xls file provided by the CBS,
	but is converted to CSV format.
	"""
	return sorted(pd.DataFrame.from_csv(location + \
					str(year) + ".csv").columns)

def get_vars(csvfile):
    """
    Returns a dictionary of correct variables for a specific year.
    """
    with open(csvfile, "rbU") as f:
        reader = reader=csv.reader(f, delimiter=';')
        return {row[0]:row[1] for row in reader}

def get_countrydata(JSON):
    minimum = min((JSON[province]["__provdata"]["min"] for province in JSON.keys()), 
                                                                key = lambda x: x[2])
    
    maximum = max((JSON[province]["__provdata"]["max"] for province in JSON.keys()), 
                                                                key = lambda x: x[2])
    
    average = np.mean([JSON[province]["__provdata"]["avg"] for province in JSON.keys()])
    
    return {"min": minimum,
            "max": maximum,
            "avg": average}


# Dictionary of correct variables for all years.
variables = {os.path.basename(fn)[:-4]: get_vars(fn)
				for fn in glob.glob('data/csv/vars/*')
				if not fn.endswith("~")}

# List of all provinces.
provinces = ["Groningen", "Friesland", "Drenthe", "Overijssel",
            "Flevoland", "Gelderland", "Utrecht", "Noord-Holland",
             "Zuid-Holland", "Zeeland", "Noord-Brabant", "Limburg"]

def get_json(var, variables, provinces, years, location = "data/csv/"):
    """
    Returns a json string per item,
    in which the data for the given years is included.

    var: string
    variables: nested dictionary
    provinces: list of Dutch provinces
    years: tuple -> ints
    location: string

    E.g.

    > get_json("aantal_mannen", variables, provinces, (2010, 2014))
    >>> {'2010': {'Drenthe':  { u'Aa en Hunze': 12685,
								u'Assen': 32605,
								...
							  },
				 'Flevoland': {u'Almere': 93510,
							   u'Dronten': 20185,
							   ...
							  }
				 },

		 '2011': {...
		 }
    """

    JSON = {}

    # For each year provided in the years tuple parameter...
    for year in xrange(years[0], years[1] + 1):
		# Create a new entry in the JSON dict.

        year = str(year)
        JSON[year] = {}

        # Retrieve correct variable names.
        item = variables[var][year]
        areades = variables["regioaanduiding"][year]
        index = variables["regionaam"][year]
        data = pd.DataFrame.from_csv(location + year + ".csv")

        # Lower all input and all columns
        # (CBS uses multicase for its columns throughout the years).
        item, areades = item.lower(), areades.lower()
        index = index.lower()
        data.columns = map(str.lower, data.columns)

        # First select only all gemeentes,
        # then filter based on given item and index.
        # This will be for all gemeentes in all provinces.
        root = data[(data[areades] == 'Gemeente') |
                    (data[areades] == 'G')].filter([item, index])
                    
        # If var is float, multiply by 100.
        if var == "gemiddelde_huishoudensgrootte" or var == "personenautos_per_huishouden":
            root[item] = root[item].apply(lambda x: float(x.replace(',', '.'))*100 if x != "x" else None)

        # DataFrame consisting of Gemeente,Provincie data.
        # Foor 2004 and 2005, assuming data is equal to 2006 data,
        # because of nonexistence of data for these years.
        try:
            gemprov = pd.DataFrame.from_csv(location +
                                            "gemprov/" + year + ".csv")
        except Exception as e:
            gemprov = pd.DataFrame.from_csv(location +
                                            "gemprov/2006.csv")

        # For each province in the provinces list parameter...
        for prov in provinces:
            # Create a list of Gemeentes in currently iterated province.
            gem = list(gemprov[(gemprov["PROVINCIE"] == prov)]
                       .filter(["GEMEENTE"]).T.columns)

            # Filter root based on this list.
            branch = root[(root[index].isin(gem))].set_index([index])

            # Lowercase all Gemeente names.
            branch.index = branch.index.str.lower()
            
            # Min, max, average for each province
            bd = branch[item].to_dict()
            values = map(lambda x: float(x), filter(lambda x: x != None and not math.isnan(x), bd.values()))
            bd.update({"__provdata": {"min": [prov, branch[item].idxmin(skipna = True), 
                                                  min(values)], 
                                      "max": [prov, branch[item].idxmax(skipna = True), 
                                                  max(values)],
                                      "avg": np.mean(values)}}
                     )
            
            # Write to JSON dictionary.
            JSON[year][prov] = json.loads(json.dumps(bd))
        
        # Min, max, average for whole country
        JSON[year]["__countrydata"] = get_countrydata(JSON[year])
        
    return JSON

In [3]:
get_json("personenautos_per_huishouden", variables, provinces, (2014, 2014))["2014"]["__countrydata"]

{'avg': 117.24437508700913,
 'max': [u'Utrecht', u'houten', 229.99999999999997],
 'min': [u'Noord-Holland', u'amsterdam', 50.0]}

In [5]:
map(lambda x: float(x), ['3', '5', 5, 3.0])

[3.0, 5.0, 5.0, 3.0]